In [144]:
import torch
import numpy as np

## torch基本使用逻辑

###### 1、Tensor是torch的基本单元，因为反向传播的计算图是根据Tensor之间的计算关系创建的。

In [145]:
w = torch.Tensor([1.0])

###### 2、一个Tensor如果在反向传播过程中需要导数的话，需要对Tensor的requries_grad属性明确赋值为True

In [146]:
w.requires_grad = True

###### 3、Tensor有两个基本的属性：data，grad。这两个属性的类型也是Tensor，但与直接对Tensor进行运算不同，使用Tensor的data和grad属性运算，不会产生计算图

In [147]:
print(type(w))
print(w)

<class 'torch.Tensor'>
tensor([1.], requires_grad=True)


In [148]:
print(type(w.data))
print(w.data)

<class 'torch.Tensor'>
tensor([1.])


In [149]:
w.grad = torch.Tensor([0.])
print(type(w.grad))
print(w.grad)

<class 'torch.Tensor'>
tensor([0.])


###### 4、Tensor有一个基本的方法：item()。用来返回Tensor的具体内容，返回的类型是int或者float或其他基本类型，意味着用item()返回的内容进行运算时，将不会产生运算图，也没有浪费内存或者溢出的风险

In [150]:
print(type(w.item()))
print(w.item())

<class 'float'>
1.0


###### 5、进行反向传播的时候只需要对最后的Tensor调用backward()功能，计算图中各个Tensor的导数会自动更新。

In [151]:
X = torch.Tensor([1.0,2.0,3.0])
Y = torch.Tensor([3.0,5.0,7.0])

def forward(w,X,Y):
    out = (Y-w*X)**2
    return torch.sum(out)

y = forward(w,X,Y)
y.backward()
print('\tw的导数为：\n',w.grad)

	w的导数为：
 tensor([-40.])


#### 6、更新权重的时候应该使用Tensor的data属性更新，而不是直接更新Tensor。
#### 原因是直接更新Tensor会形成新的计算图，注意看下方直接更新Tensor，即‘w = w-w.grad’之后w的结果，grad_fn=SubBackward0表示了这个Tensor是由一个减法运算得到的，其中0表示第一次减法，这意味着这次更新产生了计算图，如果持续通过这种方式更新，可能造成内存爆炸。
#### 相比之下，通过‘w.data = w.data - w.grad.data’的方式对属性进行更新时，更新的结果是一样的，但是没有产生计算图，只是保留了w保存导数的属性，所以应该用这种方式

In [152]:
w.data = w.data - w.grad.data
print(w)

tensor([41.], requires_grad=True)


In [142]:
w = w - w.grad
print(w)

tensor([41.], grad_fn=<SubBackward0>)


###### 7、在更新完一次权重和偏置后，需要调用zero_()方法手动将导数归零。

In [120]:
w.grad.data.zero_()
print(w.grad)

tensor([0.])


#### 8、一些其他在自动求导过程中的基本概念。来源（https://www.freesion.com/article/7646923754/）
###### 概念3:叶子结点
在计算图中，由用户自己创建的数据就叫叶子结点，比如上图的w,x,b都是我们自己生成的，也可以说不是经过某种计算得出来的数据就是叶子结点。可以用 a.is_leaf判断是不是叶子结点（a是一个tensor)

###### 概念4:grad_fn
z.grad_fn输出的是<AddBackward0 at 0x7fb73c7cd490>

tensor由某个操作获得，在pytorch每个操作的反向传播函数是已经被定义好的，比如z是由add即加操作得到的，那么z.grad_fn得到的就是add函数的反向传播函数（好像可以直接理解为求导函数）。注意我们的得到的是AddBackward0 后面有个0，说明一个计算图中可以出现很多次add，每个add的反向传播函数是不一样的。

###### 概念5:next_functions
z.grad_fn.next_functions 输出的是
((<MulBackward0 at 0x7fb73c7cd7d0>, 0L),
(<AccumulateGrad at 0x7fb73c7cdad0>, 0L))
z是由add操作得到的，那么add操作的输入是b 和 y，输出的就是b.grad_fn和y.grad_fn。

AccumulateGrad是什么？a.grad是什么？为什么梯度要置0
对于y.grad_fn我们可以知道MulBackward就是 乘 操作对应的反向传播函数。那么b我们只是一个叶子结点，是一个Tensor,他的grad_fn即Accumlate_Grad表示这个b的导数是可积累的。比如你第一次方向传播一次，我们得出b的导数为3，即a.grad为3，但是你再求导一次，就会发现a.grad为6，这就是所谓的可累加。所以在pytorch里面，每一个batch即每一次反向传播前都会把梯度下降即grad都置为0.

###### 概念6:retain_graph=True backward()

z.backward(retain_graph=True)
z.backward()表示从z求出来的是z对各个变量的导数。
retain_graph=True表示保存中间变量。比如我们计算z对w的导数发现导数就是y，注意这个y在我们上面举例的计算图中不是我们自己指定的，是中间求出来的，我们第一次z.backward()求z对w的导数会取到y的值。但是如果我们这次传播完立刻在想传播一次，那么就会报错，因为一次梯度玩会自动把中间的计算东西释放掉，也就是第二次传播时候就没有y了，除非你再前向传播一次。所以我们可以提前指定这个保证第一次传播完中间变量仍然存在。

###### 概念7:hook函数

非叶子节点的导求出来后会被释放，如果想看怎么版，可以用autograd.grad或者hook函数。